In [1]:
# Load relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings
from sklearn.cluster import KMeans 

In [14]:
## load data set
carmax_df = pd.read_csv('CaseDataRemodeled.csv')
top10_df = pd.read_csv('Top10ByIncome.csv')

In [21]:
cluster_df = carmax_df
cluster_df = cluster_df.drop('insert_num', axis=1)

## Time to Cluster - Gonna use k-means initially 

In general, we can cluster observations on the basis of the features in order to identify subgroups among the observations, or we can cluster fea- tures on the basis of the observations in order to discover subgroups among the features. In what follows, for simplicity we will discuss clustering obser- vations on the basis of the features, though the converse can be performed by simply transposing the data matrix.

In [41]:
cluster_df = carmax_df
cluster_df = cluster_df.drop('insert_num', axis=1)
cluster_df = cluster_df.drop('customer_age', axis=1)
cluster_df = cluster_df.drop('post_purchase_satisfaction', axis=1)
cluster_df.head()

False

0               sedan
1               truck
2               sedan
3               sedan
4               sedan
5               sedan
6               truck
7                 suv
8                 suv
9               sedan
10              sedan
11              sedan
12                suv
13                suv
14              sedan
15              wagon
16                van
17              truck
18              sedan
19              sedan
20                suv
21              sedan
22              sedan
23              sedan
24              truck
25              sedan
26              sedan
27              wagon
28              coupe
29              wagon
             ...     
355844          wagon
355845            suv
355846          sedan
355847          sedan
355848          sedan
355849            suv
355850          wagon
355851            suv
355852            van
355853    convertible
355854          sedan
355855          sedan
355856            suv
355857          wagon
355858    

In [24]:
# function to transform into numerical bins by normal/gaussian distribution
def categorical_transform(col):
    classnames, factorval, rank = np.unique(col, return_inverse = True,return_counts = True)
    
    temp = np.array(rank)
    temp.sort()
    temp = np.concatenate((temp[::2], temp[~np.isin(temp,temp[::2])][::-1]))

    category_count = {}
    for i in range(len(classnames)):
        category_count[classnames[i]] = rank[i]
    
    vals = [i * 5 for i in range(len(temp))]
    
    for i in range(len(temp)):
        for key, value in category_count.items():
            if value == temp[i]:
                category_count[key] = vals[i]
            
    check_list = list(col)
    for i in range(len(check_list)):
        for key, value in category_count.items():
            if check_list[i] == key:
                factorval[i] = value 
                break
    
    return category_count, factorval

In [29]:
## references of transformation for later 
references = {}

for i in cluster_df.columns:
    if cluster_df[i].dtype == 'object':
        classes = cluster_df[i]
        references[i], cluster_df[i] = categorical_transform(classes)

purchase_class


TypeError: '<' not supported between instances of 'str' and 'float'